In [13]:
import torch
from tqdm import tqdm
from gpt import GPTLanguageModel
from MoD import GPTLanguageMoD

In [20]:
batch_size = 32
block_size = 4
max_iters = 6000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 768
n_head = 6
n_layer = 6
dropout = 0.2

torch.manual_seed(1337)
device

'cpu'

In [21]:
with open('data/big_input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


In [22]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [23]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [24]:
model = GPTLanguageMoD(vocab_size=vocab_size)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

42.75472 M parameters


In [25]:
model = GPTLanguageModel(vocab_size=vocab_size)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

42.740884 M parameters


In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in tqdm(range(max_iters)):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|          | 0/6000 [00:00<?, ?it/s]

step 0: train loss 5.2804, val loss 5.2685


  4%|▍         | 233/6000 [00:34<14:25,  6.67it/s] 


KeyboardInterrupt: 

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

	– А охочешь, птешение лестные и моейщей предания, был душлив и казалось даже заметить, что он не был в своенное вцарило учасов и не опять в Желтый дом Вре-ей.

– Федор не я скажу.

– Да о Николай Петрович не отроду прос?

– Как ничего не отступается. Он состал на дворе только в Герасимоваригрорам не мог как не рассуждать от него самого: именно послала другу темноту… Целых города, да такая надела над мне небольшая тревка, слегка отбила его винов.

– Он не уговорил, но и не вылказал и со всеми сво


# Сложение

In [69]:
stoi = {str(i): i for i in range(10)}
stoi['+'] = 11
stoi['-'] = 12
stoi['='] = 13
stoi['-1'] = -99

itos = {v: k for k, v in stoi.items()}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

vocab_size = len(itos)

device = 'cpu' # в какой-то момент были траблы с гпу, но тут и на цпу норм

In [70]:
import random
def make_samples(n=batch_size):
    x_samples = []
    y_samples = []
    for i in range(n):
        a = random.randint(0, 9)
        b = random.randint(0, 9)
        c = a + b
        input_seq = str(a) + '+' + str(b) + "="
        x = encode(input_seq)
        y = [-1] * (len(input_seq) - len(str(c))) + encode([d for d in str(c)][::-1]) # Reverse the digits of c
        x_samples.append(torch.tensor(x))
        y_samples.append(torch.tensor(y))
    return x_samples, y_samples

In [71]:
def get_batch_num():
    x, y = make_samples()
    x = torch.stack(x)
    y = torch.stack(y)
    x, y = x.to(device), y.to(device)
    return x, y

In [72]:
@torch.no_grad()
def estimate_loss_num():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch_num()
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [73]:
model = GPTLanguageModel(vocab_size=vocab_size)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

42.534926 M parameters


In [87]:
max_iters = 500

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss_num()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch_num()

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 0.0101, val loss 0.0104
step 499: train loss 0.0034, val loss 0.0035


Тут еще есть интересная проблема, что нужно было учить модель еще генерить что-то по типу <EOS>, потому что ответ может быть маленьким, а max_new_tokens заставляет генерировать дальше

К сожалению не успеваю сделать до дедлайна, поэтому если можно то могу попробовать решить эту проблему + задание 5 после дедлайна

In [102]:
example = '1+3='
context = torch.tensor([encode(example)])
print(decode(m.generate(context, max_new_tokens=1)[0].tolist()))

1+3=4


In [109]:
example = '5+5='
context = torch.tensor([encode(example)])
print(decode(m.generate(context, max_new_tokens=2)[0].tolist()))

5+5=10
